In [1]:
import csv
from datetime import datetime

FMT = '%H:%M:%S.%f'   # time format

filename = 'arbitrage1.csv'
with open(filename, newline='') as f:
    reader = csv.reader(f)
    results = list(reader)

print(results[0])
len(results)

['2021-06-08 14:46:31.203918', 'USDT->ARDR->BTC->USDT', '0.091']


61

In [2]:
norm = {}   # normalized dictionary: times and profit for each exchange
for i, result in enumerate(results):
    if result[1] not in norm:  # have we seen this exchange yet?
        norm[result[1]] = [[result[0][11:22], result[2]]]
    else:
        norm[result[1]].append([result[0][11:22], result[2]])

In [3]:
results = {}
sequences = []
sequence = 0
prev_time = None
for key in norm:
    for t in norm[key]:
        if not prev_time:
            prev_time = t[0]
            continue

        if (datetime.strptime(t[0], FMT) - datetime.strptime(prev_time, FMT)).total_seconds() < 1.5:
            sequence += 1
        else:    # broken sequence
            if sequence:    # did we find a sequence?
                if sequences:    # do we already have sequences of consecutive times?
                    sequences.append([sequence, t[0], t[1]])
                else:
                    sequences = [[sequence, t[0], t[1]]]    # create a new sequence list
                sequence = 1

        prev_time = t[0]

    if sequence:   # take care of any leftover sequence
        sequences.append([sequence, t[0], t[1]])

    if sequences:
        results[key] = sequences
    sequences = []
    sequence = 0
    
results

{'USDT->BNB->TROY->USDT': [[1, '14:57:06.26', '0.0714']],
 'USDT->BTC->POND->USDT': [[1, '15:11:55.46', '0.1555'],
  [1, '15:11:55.46', '0.1555']],
 'USDT->POND->BTC->USDT': [[1, '15:35:53.35', '0.0831'],
  [1, '15:35:53.35', '0.0831']],
 'USDT->BTC->SUSD->USDT': [[5, '15:36:40.69', '0.0722'],
  [3, '15:36:41.93', '0.159']],
 'USDT->IRIS->BTC->USDT': [[1, '14:57:19.19', '0.1332']],
 'USDT->WAN->BTC->USDT': [[1, '15:01:53.48', '0.0329']],
 'USDT->COCOS->BNB->USDT': [[1, '15:01:53.48', '0.0091']],
 'USDT->WING->BNB->USDT': [[4, '15:04:16.92', '0.0029']],
 'USDT->BNB->AAVE->USDT': [[1, '15:11:04.73', '0.0706']],
 'USDT->ETH->AAVE->USDT': [[1, '15:11:04.73', '0.0465']],
 'USDT->MFT->ETH->USDT': [[1, '15:18:45.87', '0.0631']],
 'USDT->BTC->RUNE->USDT': [[1, '15:23:58.05', '0.0054']],
 'USDT->BNB->COCOS->USDT': [[1, '15:36:43.03', '0.016']],
 'USDT->BNB->STMX->USDT': [[3, '15:41:15.04', '0.0217'],
  [2, '15:41:15.74', '0.0292']],
 'USDT->ETH->KEY->USDT': [[1, '15:46:22.55', '0.228'],
  [2, '

In [4]:
trades = []
long_trades = []
for key in results:
    for trade in results[key]:
        if trade[0] > 3:
            if long_trades:
                long_trades.append(float(trade[2]))
            else:
                long_trades = [float(trade[2])]
                
        if trades:
            trades.append(float(trade[2]))
        else:
            trades = [float(trade[2])]
            
print('arbitrage %', len(long_trades) / len(trades) *100)
print('avg profit margin %', sum(long_trades) / len(long_trades))
long_trades

arbitrage % 10.0
avg profit margin % 0.03755


[0.0722, 0.0029]